In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date='2022-04-08', date_built='2016', price='936923', bedrooms='4', bathrooms='3', sqft_living='3167', sqft_lot='11733', floors='2', waterfront='1', view='76')

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT ROUND(AVG(price),2), YEAR(date) 
FROM home_sales 
WHERE bedrooms == 4 
GROUP BY YEAR(date) 
ORDER BY YEAR(date) DESC
"""
spark.sql(query).show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|           301819.44|      2021|
|           298353.78|      2020|
|            300263.7|      2019|
+--------------------+----------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT ROUND(AVG(price),2), YEAR(date_built) 
FROM home_sales 
WHERE bedrooms == 3 AND bathrooms == 3
GROUP BY YEAR(date_built) 
ORDER BY YEAR(date_built) DESC
"""
spark.sql(query).show()

+--------------------+----------------+
|round(avg(price), 2)|year(date_built)|
+--------------------+----------------+
|           292676.79|            2017|
|           290555.07|            2016|
|            288770.3|            2015|
|           290852.27|            2014|
|           295962.27|            2013|
|           293683.19|            2012|
|           291117.47|            2011|
|           292859.62|            2010|
+--------------------+----------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT ROUND(AVG(price),2), YEAR(date_built) 
FROM home_sales 
WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
GROUP BY YEAR(date_built) 
ORDER BY YEAR(date_built) DESC
"""
spark.sql(query).show()

+--------------------+----------------+
|round(avg(price), 2)|year(date_built)|
+--------------------+----------------+
|           280317.58|            2017|
|            293965.1|            2016|
|           297609.97|            2015|
|           298264.72|            2014|
|           303676.79|            2013|
|           307539.97|            2012|
|           276553.81|            2011|
|           285010.22|            2010|
+--------------------+----------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
query = """
SELECT ROUND(AVG(price),2), view 
FROM home_sales 
GROUP BY view HAVING ROUND(AVG(price),2) >= 350000
"""
spark.sql(query).show()
uncached_time = time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           788128.21|  51|
|           798684.82|  54|
|           750537.94|  69|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            791453.0|  59|
|          1056336.74|  85|
|           733780.26|  52|
|            775651.1|  71|
|          1053739.33|  98|
|          1061201.42|  99|
|          1017815.92|  96|
|           1026669.5| 100|
|           695865.58|  70|
|           746877.59|  61|
|          1114042.94|  75|
|          1080649.37|  78|
|          1107839.15|  89|
|          1076205.56|  77|
+--------------------+----+
only showing top 20 rows

--- 0.2986907958984375 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()
query = """
SELECT ROUND(AVG(price),2), view 
FROM home_sales 
GROUP BY view HAVING ROUND(AVG(price),2) >= 350000
"""
spark.sql(query).show()
cached_time = time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s second reduction ---" % (uncached_time - cached_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           788128.21|  51|
|           798684.82|  54|
|           750537.94|  69|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            791453.0|  59|
|          1056336.74|  85|
|           733780.26|  52|
|            775651.1|  71|
|          1053739.33|  98|
|          1061201.42|  99|
|          1017815.92|  96|
|           1026669.5| 100|
|           695865.58|  70|
|           746877.59|  61|
|          1114042.94|  75|
|          1080649.37|  78|
|          1107839.15|  89|
|          1076205.56|  77|
+--------------------+----+
only showing top 20 rows

--- 0.18101000785827637 seconds ---
--- 0.11717963218688965 second reduction ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.parquet('home_parquet', mode='overwrite')

Py4JJavaError: An error occurred while calling o53.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputComm

In [13]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet('home_parquet')

Py4JJavaError: An error occurred while calling o55.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(Tr

In [14]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_home_sales')

NameError: name 'parquet_df' is not defined

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 
start_time = time.time()
query = """
SELECT ROUND(AVG(price),2), view 
FROM parquet_home_sales 
GROUP BY view HAVING ROUND(AVG(price),2) >= 350000
"""
spark.sql(query).show()
parquet_time = time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s second reduction ---" % (uncached_time - parquet_time))

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `parquet_home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 5;
'UnresolvedHaving ('ROUND('AVG('price), 2) >= 350000)
+- 'Aggregate ['view], [unresolvedalias('ROUND('AVG('price), 2), None), 'view]
   +- 'UnresolvedRelation [parquet_home_sales], [], false


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False